## TensorFlow 라이브러리

- 대표적인 딥러닝 라이브러리
- caffe, pytorch 등 다양하다.

## Keras 라이브러리

- 딥러닝 모델을 쉽게 구현하도록 해주는 딥러닝 상위 레벨 라이브러리
- Keras가 구동되려면 TensorFlow가 설치되어 있어야 함. (TensorFlow 2.0부터 내장되어있음.)

In [1]:
import tensorflow as tf

In [2]:
import keras

---

## 폐암 수술 환자의 생존율 예측

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('./thoraric_surgery.csv')
data_set = df.values
data_set

array([[293.  ,   1.  ,   3.8 , ...,   0.  ,  62.  ,   0.  ],
       [  1.  ,   2.  ,   2.88, ...,   0.  ,  60.  ,   0.  ],
       [  8.  ,   2.  ,   3.19, ...,   0.  ,  66.  ,   1.  ],
       ...,
       [406.  ,   6.  ,   5.36, ...,   0.  ,  62.  ,   0.  ],
       [ 25.  ,   8.  ,   4.32, ...,   0.  ,  58.  ,   1.  ],
       [447.  ,   8.  ,   5.2 , ...,   0.  ,  49.  ,   0.  ]])

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_set, test_size = 0.2)

# train[행, 열] -> train[:, 0:17] => 모든행, 0 - 17 열 슬라이싱
X_train = train[:, 0:17]
Y_train = train[:, 17]

X_test = test[:, 0:17]
Y_test = test[:, 17]

### 딥러닝 구조 : 입력층, 은닉층, 출력층 설정하기
- `Sequential()`로 딥러닝 구조의 모델 객체를 생성
- `add()` 메서드로 층을 추가
- 각각의 층은 `Dense()` 객체를 통해 구체적인 구조가 결정됨.
    - Keras는 입력층을 따로 만들지 않고 첫 번째 Dense가 은닉층 + 입력층의 역할을 겸함
    - 인수 : 각 층의 노드수, input_dim = 입력 값 수, activation = 사용할 활성화 함수

In [5]:
from keras.models import Sequential
from keras.layers import Dense

# 딥러닝 구조 모델인 Sequential 객체, model 생성.
model = Sequential()
# 2개의 층을 추가
model.add(Dense(30, input_dim = 17, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

<img src = "./example_img.png">

### 모델 컴파일 하기 : complile() 메소드

모델을 더 효과적으로 구현할 학습 환경을 설정하여 완성

- loss = 한 번 신경망이 실행될 때마다 오차 값을 추적하는 함수.
- optimizer = 오차를 어떻게 줄여 나갈지 정하는 함수.
- metrics = 학습과 테스트 단계에서 모델을 평가하기 위한 성능 기준을 설정.

<img src ="model_compile.png">


In [6]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])

### 모델 실행하기

`fit()` 메소드 : 학습 데이터를 가지고 학습

- epochs(에포크) = 각 샘플(데이터에서 한 행)을 반복하여 실행하는 함수.
- batch_size = 샘플을 한 번에 처리하는 개수

In [7]:
model.fit(X_train, Y_train, epochs = 30, batch_size = 10)

Epoch 1/30
38/38 [==============================] - 0s 472us/step - loss: 0.1554 - accuracy: 0.8351
Epoch 2/30
38/38 [==============================] - 0s 446us/step - loss: 0.1425 - accuracy: 0.8564
Epoch 3/30
38/38 [==============================] - 0s 472us/step - loss: 0.1430 - accuracy: 0.8511
Epoch 4/30
38/38 [==============================] - 0s 472us/step - loss: 0.1429 - accuracy: 0.8537
Epoch 5/30
38/38 [==============================] - 0s 446us/step - loss: 0.1420 - accuracy: 0.8537
Epoch 6/30
38/38 [==============================] - 0s 472us/step - loss: 0.1424 - accuracy: 0.8511
Epoch 7/30
38/38 [==============================] - 0s 446us/step - loss: 0.1418 - accuracy: 0.8511
Epoch 8/30
38/38 [==============================] - 0s 446us/step - loss: 0.1425 - accuracy: 0.8564
Epoch 9/30
38/38 [==============================] - 0s 446us/step - loss: 0.1443 - accuracy: 0.8537
Epoch 10/30
38/38 [==============================] - 0s 472us/step - loss: 0.1414 - accuracy: 0.8564

### 모델 평가 및 사용

`evaluate()` 메소드 : 생성한 딥러닝 모델의 정확도 점검

In [8]:
model.evaluate(X_test, Y_test)

3/3 [==============================] - 0s 665us/step - loss: 0.1813 - accuracy: 0.8191


[0.181261345744133, 0.8191489577293396]

`predict_classes()` 메소드 : 생성한 딥러닝 모델을 이용하여 예측

In [9]:
from sklearn import metrics

prediction = model.predict_classes(X_test)
ac_score = metrics.accuracy_score(Y_test, prediction)

print("Accuracy : ", ac_score)
comparison = pd.DataFrame({'Prediction':prediction.ravel(), 'Ground-Truth:':Y_test.astype(int)})
comparison

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Accuracy :  0.8191489361702128


,Prediction,Ground-Truth:
0,0,0
1,0,0
2,0,1
3,0,1
4,0,0
...,...,...
89,0,0
90,0,0
91,0,0
92,0,0


### TensorFlow 와 Keras를 이용한 딥러닝 순서

1. Sequential 모형 클래스 객체 생성
2. add 메서드로 레이어 추가.
    - 입력단부터 각층은 Dense함수로 추가. Dense(노드개수, input_dim = 입력크기, activation 함수)
3. compile 메서드로 모형 완성.
    - loss = 비용함수, optimizer = 최적화 알고리즘, metrics = 성능 기준 설정.
4. fit 메서드로 학습.
    - epoch = 재사용 횟수, batch_size = 배치크기 설정.
5. evaluate 메서드로 모델 평가
6. predict 메서드로 예측.